## 准备数据

In [ ]:
import os
import numpy as np
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers, optimizers, datasets

# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

# def mnist_dataset():
#     (x, y), (x_test, y_test) = datasets.mnist.load_data()
#     #normalize
#     x = x/255.0
#     x_test = x_test/255.0
    
#     return (x, y), (x_test, y_test)
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

def mnist_dataset(batch_size=64):
    # 定义数据预处理步骤：转换为Tensor以及标准化
    transform = transforms.Compose([
        transforms.ToTensor(),  # 将图像转换为Tensor，并且数值范围归一化到[0, 1]
        transforms.Normalize((0.1307,), (0.3081,)),  # 使用MNIST数据集的均值和标准差进行标准化
    ])

    # 下载训练数据集并进行预处理
    train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # 下载测试数据集并进行预处理
    test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader



## Demo numpy based auto differentiation

In [45]:
import numpy as np

class Matmul:
    def __init__(self):
        self.mem = {}
        
    def forward(self, x, W):
        h = np.matmul(x, W)
        self.mem={'x': x, 'W':W}
        return h
    
    def backward(self, grad_y):
        '''
        x: shape(N, d)
        w: shape(d, d')
        grad_y: shape(N, d')
        '''
        x = self.mem['x']
        W = self.mem['W']
        
        ####################
        '''计算矩阵乘法的对应的梯度'''
        ####################
        grad_x = np.matmul(grad_y, W.T)
        grad_W = np.matmul(x.T, grad_y)
        return grad_x, grad_W


class Relu:
    def __init__(self):
        self.mem = {}
        
    def forward(self, x):
        self.mem['x']=x
        return np.where(x > 0, x, np.zeros_like(x))
    
    def backward(self, grad_y):
        '''
        grad_y: same shape as x
        '''
        ####################
        '''计算relu 激活函数对应的梯度'''
        ####################
        grad_x = np.where(self.mem['x'] > 0, grad_y, np.zeros_like(grad_y))
        return grad_x
    


class Softmax:
    '''
    softmax over last dimention
    '''
    def __init__(self):
        self.epsilon = 1e-12
        self.mem = {}
        
    def forward(self, x):
        '''
        x: shape(N, c)
        '''
        # 减去最大值以防止溢出
        x_exp = np.exp(x - np.max(x, axis=-1, keepdims=True))
        partition = np.sum(x_exp, axis=1, keepdims=True)
        out = x_exp/(partition+self.epsilon)
        
        self.mem['out'] = out
        self.mem['x_exp'] = x_exp
        return out
    
    def backward(self, grad_y):
        '''
        grad_y: same shape as x
        '''
        # s = self.mem['out']
        # sisj = np.matmul(np.expand_dims(s,axis=2), np.expand_dims(s, axis=1)) # (N, c, c)
        # g_y_exp = np.expand_dims(grad_y, axis=1)
        # tmp = np.matmul(g_y_exp, sisj) #(N, 1, c)
        # tmp = np.squeeze(tmp, axis=1)
        # tmp = -tmp+grad_y*s 
        # return tmp

        s = self.mem['out']  # Softmax 输出
        batch_size, num_classes = s.shape
        
        # 初始化梯度矩阵
        grad_x = np.zeros_like(s)
    
        for n in range(batch_size):
            s_diag = np.diag(s[n])  # 对角矩阵
            s_matrix = np.outer(s[n], s[n])  # 外积矩阵
            
            # 计算 Jacobian 矩阵
            jacobian = s_diag - s_matrix
            
            # 计算当前样本的梯度
            grad_x[n] = np.dot(grad_y[n], jacobian)
    
        return grad_x
    
class Log:
    '''
    softmax over last dimention
    '''
    def __init__(self):
        self.epsilon = 1e-12
        self.mem = {}
        
    def forward(self, x):
        '''
        x: shape(N, c)
        '''
        out = np.log(x+self.epsilon)
        
        self.mem['x'] = x
        return out
    
    def backward(self, grad_y):
        '''
        grad_y: same shape as x
        '''
        x = self.mem['x']
        
        return 1./(x+1e-12) * grad_y
    


## Gradient check

In [46]:
# import tensorflow as tf

# x = np.random.normal(size=[5, 6])
# W = np.random.normal(size=[6, 4])
# aa = Matmul()
# out = aa.forward(x, W) # shape(5, 4)
# grad = aa.backward(np.ones_like(out))
# print (grad)

# with tf.GradientTape() as tape:
#     x, W = tf.constant(x), tf.constant(W)
#     tape.watch(x)
#     y = tf.matmul(x, W)
#     loss = tf.reduce_sum(y)
#     grads = tape.gradient(loss, x)
#     print (grads)

# import tensorflow as tf

# x = np.random.normal(size=[5, 6])
# aa = Relu()
# out = aa.forward(x) # shape(5, 4)
# grad = aa.backward(np.ones_like(out))
# print (grad)

# with tf.GradientTape() as tape:
#     x= tf.constant(x)
#     tape.watch(x)
#     y = tf.nn.relu(x)
#     loss = tf.reduce_sum(y)
#     grads = tape.gradient(loss, x)
#     print (grads)

# import tensorflow as tf
# x = np.random.normal(size=[5, 6], scale=5.0, loc=1)
# label = np.zeros_like(x)
# label[0, 1]=1.
# label[1, 0]=1
# label[1, 1]=1
# label[2, 3]=1
# label[3, 5]=1
# label[4, 0]=1
# print(label)
# aa = Softmax()
# out = aa.forward(x) # shape(5, 6)
# grad = aa.backward(label)
# print (grad)

# with tf.GradientTape() as tape:
#     x= tf.constant(x)
#     tape.watch(x)
#     y = tf.nn.softmax(x)
#     loss = tf.reduce_sum(y*label)
#     grads = tape.gradient(loss, x)
#     print (grads)

# import tensorflow as tf

# x = np.random.normal(size=[5, 6])
# aa = Log()
# out = aa.forward(x) # shape(5, 4)
# grad = aa.backward(label)
# print (grad)

# with tf.GradientTape() as tape:
#     x= tf.constant(x)
#     tape.watch(x)
#     y = tf.math.log(x)
#     loss = tf.reduce_sum(y*label)
#     grads = tape.gradient(loss, x)
#     print (grads)

# Final Gradient Check

In [ ]:
# import tensorflow as tf
import torch.nn.functional as F

x = np.random.normal(size=[5, 6])

label = np.zeros_like(x)
label[0, 1]=1.
label[1, 0]=1
label[2, 3]=1
label[3, 5]=1
label[4, 0]=1

W1 = np.random.normal(size=[6, 5])
W2 = np.random.normal(size=[5, 6])

mul_h1 = Matmul()
mul_h2 = Matmul()
relu = Relu()
softmax = Softmax()
log = Log()

h1 = mul_h1.forward(x, W1) # shape(5, 4)
h1_relu = relu.forward(h1)
h2 = mul_h2.forward(h1_relu, W2)
h2_soft = softmax.forward(h2)
h2_log = log.forward(h2_soft)


h2_log_grad = log.backward(label)
h2_soft_grad = softmax.backward(h2_log_grad)
h2_grad, W2_grad = mul_h2.backward(h2_soft_grad)
h1_relu_grad = relu.backward(h2_grad)
h1_grad, W1_grad = mul_h1.backward(h1_relu_grad)

# 打印自定义函数计算的梯度
print("Gradient of log softmax output in custom function:")
print(h2_log_grad)
print('--'*20)


# 转换为PyTorch张量，并设置requires_grad=True以便计算梯度
x_torch = torch.tensor(x, dtype=torch.float32, requires_grad=True)
W1_torch = torch.tensor(W1, dtype=torch.float32, requires_grad=True)
W2_torch = torch.tensor(W2, dtype=torch.float32, requires_grad=True)
label_torch = torch.tensor(label, dtype=torch.float32)

# 前向传播
h1 = torch.matmul(x_torch, W1_torch)
h1_relu = F.relu(h1)
h2 = torch.matmul(h1_relu, W2_torch)
prob = F.softmax(h2, dim=1)
prob.retain_grad()  # 保留 prob 的梯度
log_prob = torch.log(prob)

# 计算损失
loss = -torch.sum(label_torch * log_prob)

# 反向传播
loss.backward()

# 打印Softmax输出相对于loss的梯度
print("Gradient of log softmax output in PyTorch:")
print(prob.grad)  # 现在可以打印 prob 的梯度

Gradient of log softmax output in custom function:
[[0.00000000e+00 2.37817110e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.05548270e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 2.64069518e+02
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 1.15386901e+05]
 [1.17846490e+01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
----------------------------------------
Gradient of log softmax output in PyTorch:
tensor([[-0.0000e+00, -2.3782e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00,
         -0.0000e+00],
        [-1.0555e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00,
         -0.0000e+00],
        [-0.0000e+00, -0.0000e+00, -0.0000e+00, -2.6407e+02, -0.0000e+00,
         -0.0000e+00],
        [-0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00,
         -1.1539e+05],
 

## 建立模型

In [ ]:

class myModel:
    def __init__(self):
        
        self.W1 = np.random.normal(size=[28*28+1, 100])
        self.W2 = np.random.normal(size=[100, 10])
        
        self.mul_h1 = Matmul()
        self.mul_h2 = Matmul()
        self.relu = Relu()
        self.softmax = Softmax()
        self.log = Log()
        
        
    def forward(self, x):
        x = x.reshape(-1, 28*28)
        bias = np.ones(shape=[x.shape[0], 1])
        x = np.concatenate([x, bias], axis=1)

        self.h1 = self.mul_h1.forward(x, self.W1) # shape(5, 4)
        self.h1_relu = self.relu.forward(self.h1)
        self.h2 = self.mul_h2.forward(self.h1_relu, self.W2)
        self.h2_soft = self.softmax.forward(self.h2)
        self.h2_log = self.log.forward(self.h2_soft)
            
    def backward(self, label):
        # 这里的label是one-hot编码的
        self.h2_log_grad = self.log.backward(-label)
        self.h2_soft_grad = self.softmax.backward(self.h2_log_grad)
        self.h2_grad, self.W2_grad = self.mul_h2.backward(self.h2_soft_grad)
        self.h1_relu_grad = self.relu.backward(self.h2_grad)
        self.h1_grad, self.W1_grad = self.mul_h1.backward(self.h1_relu_grad)
        
model = myModel()


## 计算 loss

In [ ]:
def compute_loss(log_prob, labels):
    # log_prob: (batch_size, num_classes)
    # labels: (batch_size,)
    # 将lables转化为one-hot编码
    if len(labels.shape) == 1:
        labels = np.eye(log_prob.shape[1])[labels]
    return np.mean(np.sum(-log_prob*labels, axis=1))
    

def compute_accuracy(log_prob, labels):
    predictions = np.argmax(log_prob, axis=1)
    # 如果labels是one-hot编码，则需要将其转化为类别索引
    if len(labels.shape) == 1:
        truth = labels
    else:
        truth = np.argmax(labels, axis=1)
    return np.mean(predictions==truth)

def train_one_step(model, x, y, lr = 1e-3):
    model.forward(x)
    loss = compute_loss(model.h2_log, y)
    accuracy = compute_accuracy(model.h2_log, y)
    model.backward(np.eye(10)[y])
    model.W1 -= lr* model.W1_grad
    model.W2 -= lr* model.W2_grad
    return loss, accuracy

def test(model, x, y):
    model.forward(x)
    loss = compute_loss(model.h2_log, y)
    accuracy = compute_accuracy(model.h2_log, y)
    return loss, accuracy

## 实际训练

In [91]:
train_data, test_data = mnist_dataset() # 返回dataloader

for epoch in range(50):
    epoch_loss = 0
    correct = 0
    total = 0
    for batch_idx, (data, target) in enumerate(train_data):
        loss, batch_correct = train_one_step(model, data.numpy(), target.numpy())
        epoch_loss += loss
        correct += batch_correct
        total += 1
    
    accuracy = correct / total
    print('epoch', epoch, ': loss', epoch_loss/(batch_idx+1), '; accuracy', accuracy)



epoch 0 : loss 9.738585356704895 ; accuracy 0.6414245735607675
epoch 1 : loss 9.456768159369659 ; accuracy 0.6533182302771855
epoch 2 : loss 9.334014973113167 ; accuracy 0.6581156716417911
epoch 3 : loss 8.833554323883767 ; accuracy 0.67573960554371
epoch 4 : loss 7.151023959912273 ; accuracy 0.7355910181236673
epoch 5 : loss 6.887794766918164 ; accuracy 0.7455690298507462
epoch 6 : loss 6.781072551651893 ; accuracy 0.7500333155650319
epoch 7 : loss 6.693576861992779 ; accuracy 0.7533148987206824
epoch 8 : loss 6.596792246148252 ; accuracy 0.7570462420042644
epoch 9 : loss 6.5173988344141 ; accuracy 0.7600279850746269
epoch 10 : loss 6.446938419606185 ; accuracy 0.7631763059701493
epoch 11 : loss 6.40203710085113 ; accuracy 0.7645922174840085
epoch 12 : loss 6.363427414285798 ; accuracy 0.7664412313432836
epoch 13 : loss 6.3305246673054185 ; accuracy 0.767507329424307
epoch 14 : loss 6.303634530168312 ; accuracy 0.7684734808102346
epoch 15 : loss 6.2568607901429765 ; accuracy 0.7705890

In [92]:
total_loss = 0.0
total_accuracy = 0.0
count = 0
for data, label in test_data:
    loss, accuracy = test(model, data.numpy(), label.numpy())
    total_loss += loss
    total_accuracy += accuracy
    count += 1

print("Test loss:", total_loss / count)
print("Test accuracy:", total_accuracy / count)


Test loss: 6.052110989899218
Test accuracy: 0.7783638535031847
